In [163]:
import random
import torch
from d2l import torch as d2l


# 合成数据集
def synthetic_data(w, b, num_examples):
    """
    生成y=Xw+b噪声，正态分布  0的均值，1的标准差
    X是features  y是labels
    X是10x2的矩阵,w是2x1的向量，b是标量， X*w为10x1的向量，然后b用广播机制加上去(向量内每个元素加上b的值)
    """
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    # y再加上噪声
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape(-1, 1)


def data_iter(batch_size, features, labels):
    num_examples = len(features)
    # 生成下标列表  0-1000
    indices = list(range(num_examples))
    # 打乱列表
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        # 取出每隔batch_size大小的间隔下标列表
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num_examples)])
        # print(batch_indices)
        yield features[batch_indices], labels[batch_indices]


# 定义模型 b是标量使用广播机制相加到每个向量的分量
def linreg(X, w, b):
    """线性回归模型"""
    return torch.matmul(X, w) + b


# 定义损失函数 y_hat预测值  y真实值
def squared_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2


# 定义优化算法  params:模型参数集合 lr:学习速率  batch_size:批量大小
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    # 这里不需要计算梯度，只是用到了梯度，所以no_grad。训练的时候会计算梯度
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            # pytorch下划线结尾方法代表原地操作
            param.grad.zero_()


"""合成训练数据"""

# 合成数据用到的真实w和b
true_w = torch.tensor([2, -3.4])
true_b = 4.2
# 根据实际w和b生成1000条数据
features, labels = synthetic_data(true_w, true_b, 1000)

# 画出features的第一个特征和labels的散点图之间的线性关系
# d2l.set_figsize()
# d2l.plt.scatter(features[:, 0].detach().numpy(), labels.detach().numpy(), 1)

"""训练参数准备"""

lr = 0.03
num_epochs = 3
batch_size = 10

# 初始化模型参数  正态分布w，偏置0
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


net = linreg
loss = squared_loss


for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        # print(X.shape,w.shape,b.shape)
        y_hat = net(X, w, b)
        # print(y_hat.shape,y.shape)
        # print(y_hat)
        # print(y)
        l = loss(y_hat, y)
        # l的形状是(batch_size,1),而不是一个标量，所以计算梯度需要
        # 将l内所有的元素加到一起，并以此计算关于[w,b]的梯度
        l.sum().backward()
        # 使用参数的梯度更新参数
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        # 输出每一圈的平均loss
        train_l = loss(net(features, w, b), labels)
        print(f"epoch{epoch+1},loss {float(train_l.mean()):f}")

print(true_w)
print(w)
print(f"w的估计误差:{true_w-w.reshape(true_w.shape)}")
print(f"b的估计误差:{true_b-b}")


epoch1,loss 0.033962
epoch2,loss 0.000121
epoch3,loss 0.000048
tensor([ 2.0000, -3.4000])
tensor([[ 1.9996],
        [-3.3998]], requires_grad=True)
w的估计误差:tensor([ 0.0004, -0.0002], grad_fn=<SubBackward0>)
b的估计误差:tensor([0.0011], grad_fn=<RsubBackward1>)
